In [ ]:
# ===== Pacotes / Packages =====
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error, r2_score, roc_curve, auc
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Fixar seed para reprodutibilidade / Fix seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# ===== Carregar dados .xlsx / Load data .xlsx =====
df = pd.read_excel("NAME.xlsx")
print(df.head())

In [ ]:
# ===== Carregar dados .csv / Load data .csv =====
df = pd.read_csv("NAME.csv", sep=',') # especifica o separador / specifies the separator
print(df.head())

In [ ]:
# Separar features (X) e target (y) / Separate features (X) and target (y)
X = df.iloc[:, 1:].values   # todas as colunas menos a primeira / all columns except the first
y = df.iloc[:, 0].values    # apenas a primeira coluna / just the first column

# Se 'y' contiver strings, codificar os rótulos / If 'y' contains strings, encode the labels
if y.dtype == 'object': # ou use isinstance(y[0], str) se preferir verificar um elemento / or use isinstance(y[0], str) if you prefer to check one element
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
else:
    y_encoded = y

# Transformar y em categorias (one-hot encoding) / Transform y into categories (one-hot encoding)
# Use y_encoded para one-hot encoding / Use y_encoded for one-hot encoding
y_cat = to_categorical(y_encoded)

# Dividir em treino e teste / Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y_cat, test_size=0.2, random_state=42, stratify=y_encoded # stratify com y_encoded
)

# Normalizar dados / Normalize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Best Fit - Classificação/Classification
Define o número de neurônios na camada oculta e o número de épocas ideal com base no MSE. Utiliza 'cross entropy' como função de perda / Sets the number of neurons in the hidden layer and the optimal number of epochs based on the MSE. Uses cross entropy as the loss function.

In [ ]:
# ===== Função para criar o modelo / Function to create the model =====
def criar_modelo(num_neuronios, input_dim, output_dim):
    model = Sequential([
        Dense(num_neuronios, input_dim=input_dim, activation="relu"),
        Dense(output_dim, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
# ===== Avaliação combinada: neurônios X épocas / Combined evaluation: neurons X epochs =====

neuron_range = [2, 4, 8, 16, 32, 64] # quantidade de neurônios a ser testados / number of neurons to be tested
epoch_range = list(range(5, 105, 5))  # de 'início' a 'final' com 'passo' / from 'start' to 'end' with 'step'
mse_results = {n: [] for n in neuron_range}
melhor_mse = float("inf")
melhor_comb = (None, None)

for n in neuron_range:
    print(f"\n>> Neurons: {n}")
    for ep in epoch_range:
        model = criar_modelo(n, X.shape[1], y_cat.shape[1])
        model.fit(
            X_train, y_train,
            validation_split=0.2,
            epochs=ep,
            batch_size=16,
            verbose=0
        )
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        mse_results[n].append(mse)

        print(f"   Épocas: {ep} → MSE: {mse:.4f}")

        if mse < melhor_mse:
            melhor_mse = mse
            melhor_comb = (n, ep)

# ===== Plot MSE vs Épocas / Plotar MSE vs Épocas =====
plt.figure(figsize=(10,6))
for n in neuron_range:
    plt.plot(epoch_range, mse_results[n], marker='o', label=f'{n} neurons')

plt.title("Mean Squared Error (MSE) vs Epochs")
plt.xlabel("Epochs")
plt.ylabel("MSE on the test set")
plt.legend(title="Hidden layer")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# ===== Resultado melhor combinação / Best combination result =====
melhor_n, melhor_ep = melhor_comb
print(f"\nBest configuration: {melhor_n} neurons and {melhor_ep} epochs (MSE: {melhor_mse:.4f})")

In [ ]:
# ===== Treinar modelo final com melhor combinação / Train final model with best combination =====
model = criar_modelo(melhor_n, X.shape[1], y_cat.shape[1])
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=melhor_ep,
    batch_size=16,
    verbose=1
)

# Avaliar modelo final / Evaluate final model
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest accuracy: {acc:.2f}")

In [ ]:
# Plotting training history for classification / Traçando o histórico de treinamento para classificação
plt.figure(figsize=(12, 5))

# Plot Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Loss Curve (Cross Entropy) - Classification')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Accuracy Curve - Classification')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# ===== Mostrar e salvar pesos aprendidos / Show and save learned weights =====
for i, layer in enumerate(model.layers):
    weights, biases = layer.get_weights()
    layer_name = layer.name

    print(f"\nLayer {i+1} - {layer_name}")
    print("Weights (W):")
    print(weights)
    print("Biases (b):")
    print(biases)

    # Salvar Pesos / Save Weights
    weights_df = pd.DataFrame(weights)
    weights_csv_filename = f"weights_{layer_name}.csv"
    weights_excel_filename = f"weights_{layer_name}.xlsx"

    weights_df.to_csv(weights_csv_filename, index=False)
    weights_df.to_excel(weights_excel_filename, index=False)
    print(f"Layer weights {layer_name} saved in {weights_csv_filename} and {weights_excel_filename}")

    # Salvar Biases (verificar se a camada tem biases) / Save Biases (check if the layer has biases)
    if len(biases) > 0:
        biases_df = pd.DataFrame(biases, columns=['Bias'])
        biases_csv_filename = f"biases_{layer_name}.csv"
        biases_excel_filename = f"biases_{layer_name}.xlsx"

        biases_df.to_csv(biases_csv_filename, index=False)
        biases_df.to_excel(biases_excel_filename, index=False)
        print(f"Layer biases {layer_name} saved in {biases_csv_filename} and {biases_excel_filename}")
    else:
        print(f"Layer {layer_name} has no biases.")

# Classificação / Classification

In [ ]:
# ===== Definir modelo MLP / Define MLP Model =====
neurons = 2  # número de neurônios na camada oculta / number of neurons in the hidden layer
model = Sequential([
    Dense(neurons, input_dim=X.shape[1], activation="relu"), #input_dim=X.shape[1] diz à rede quantas entradas cada neurônio da primeira camada deve esperar, só é necessário na primeira camada / input_dim=X.shape[1] tells the network how many inputs each neuron in the first layer should expect, only needed in the first layer.
    Dense(y_cat.shape[1], activation="softmax") # camada de saida / output layer
])

In [ ]:
# Compilar modelo / compile model
model.compile(optimizer="adam", loss="mse", metrics=["accuracy"])

# Treinar modelo / Train model
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=16,
    verbose=1
)

# Avaliar modelo / Evaluate model
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {acc:.2f}")

In [ ]:
# ===== Matriz de confusão / Confusion matrix =====
# Previsões / Predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Matriz / Matrix
cm = confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=np.unique(y),
            yticklabels=np.unique(y))
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion matrix")
plt.show()

# ===== Relatório de classificação / classification report =====
print("Classification report:\n")
print(classification_report(y_true, y_pred_classes, target_names=[f"Class {c}" for c in np.unique(y)]))

In [ ]:
# Use curvas individuais (multiclass ROC) quando quiser analisar o desempenho do modelo em cada classe separadamente. / Use curvas individuais (ROC multiclasse) quando quiser analisar o desempenho do modelo em cada classe separadamente.

# Fit LabelEncoder to the original unique classes / Ajustar LabelEncoder às classes originais exclusivas
label_encoder = LabelEncoder()
label_encoder.fit(np.unique(y))

# Compute ROC curve and ROC area for each class / Calcular a curva ROC e a área ROC para cada classe
fpr = dict()
tpr = dict()
roc_auc = dict()

unique_classes = np.unique(y)  # Use original unique classes for labeling / Use classes originais exclusivas para rotulagem
n_classes = len(unique_classes)

for i in range(n_classes):
    class_value = unique_classes[i]
    # Use label_encoder to get the correct index for the current class / Use label_encoder para obter o índice correto para a classe atual
    class_index = label_encoder.transform([class_value])[0]

    # Check if the current class is present in the test set / Verifique se a classe atual está presente no conjunto de teste
    if np.sum(y_test[:, class_index]) > 0:
        fpr[i], tpr[i], _ = roc_curve(y_test[:, class_index], y_pred[:, class_index])
        roc_auc[i] = auc(fpr[i], tpr[i])
    else:
        # If a class is not in the test set, set AUC to NaN and skip plotting its curve / Se uma classe não estiver no conjunto de teste, defina AUC como NaN e pule a plotagem de sua curva
        roc_auc[i] = np.nan
        print(f"Warning: Class {class_value} has no positive samples in the test set. Skipping ROC curve for this class.")


# Plot ROC curves / Traçar curvas ROC
plt.figure(figsize=(8, 6))
colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']
for i in range(n_classes):
    # Only plot if the AUC was calculated (class was in the test set) / Somente plote se a AUC foi calculada (a classe estava no conjunto de teste)
    if not np.isnan(roc_auc[i]):
        # Ensure there are enough colors / Certifique-se de que há cores suficientes
        color = colors[i % len(colors)]
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                 label='ROC curve of class {0} (area = {1:0.2f})'.format(unique_classes[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multiclass ROC Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Use macro-average ROC quando quiser uma visão geral do desempenho do modelo em todas as classes, sem considerar desequilíbrio entre elas. / Use ROC macro-médio quando quiser uma visão geral do desempenho do modelo em todas as classes, sem considerar desequilíbrio entre elas.

# Calculate macro-average ROC curve and AUC / Calcular a curva ROC macromédia e a AUC
# First aggregate all false positive rates / Primeiro, agregue todas as taxas de falsos positivos
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes) if not np.isnan(roc_auc[i])]))

# Then interpolate all ROC curves at this points / Em seguida, interpole todas as curvas ROC nesses pontos
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    if not np.isnan(roc_auc[i]):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Average it and compute AUC / Calcule a média e a AUC
mean_tpr /= sum([not np.isnan(roc_auc[i]) for i in range(n_classes)]) # Divide by the number of classes that were in the test set/ Divida pelo número de classes que estavam no conjunto de teste

macro_roc_auc = auc(all_fpr, mean_tpr)

# Plot macro-average ROC curve
plt.figure(figsize=(8, 6))
plt.plot(all_fpr, mean_tpr, color='red', linestyle='-', linewidth=2,
         label='Macro-average ROC curve (area = {0:0.2f})'.format(macro_roc_auc))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multiclass ROC Curve (Macro-average)')
plt.legend(loc="lower right")
plt.show()

# Best Fit - Regressão / Regression
Define o número de neurônios na camada oculta e o número de épocas ideal com base no MSE. Utiliza 'Mean Squared Error' como função de perda / Set the number of neurons in the hidden layer and the optimal number of epochs based on the MSE. Use 'Mean Squared Error' as the loss function.

In [ ]:
target_column_names = ['feature1', 'feature2'] # Change to a list of your desired target columns / Alterar para uma lista de suas colunas de destino desejadas

# Check if all target columns exist in the DataFrame / Verifique se todas as colunas de destino existem no DataFrame
if all(col in df.columns for col in target_column_names):
    y_reg = df[target_column_names].values
    # Define features for regression, excluding the classification target and regression targets / Definir recursos para regressão, excluindo o alvo de classificação e os alvos de regressão
    # Assuming the first column is the classification target / Supondo que a primeira coluna seja o alvo da classificação
    feature_columns_reg = [col for col in df.columns if col not in [df.columns[0]] + target_column_names]
    X_reg = df[feature_columns_reg].values

else:
    missing_cols = [col for col in target_column_names if col not in df.columns]
    print(f"Columns {missing_cols} not found in the DataFrame. Please check the column names.")
    y_reg = None # Set y_reg to None if columns are missing / Defina y_reg como Nenhum se colunas estiverem faltando
    X_reg = None # Also set X_reg to None if columns are missing / Defina também X_reg como Nenhum se colunas estiverem faltando


# Split data for regression only if target columns were found / Dividir dados para regressão somente se colunas de destino forem encontradas
if y_reg is not None and X_reg is not None:
    X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
        X_reg, y_reg, test_size=0.2, random_state=42
    )

    # Normalize data for regression / Normalizar dados para regressão
    scaler_reg = StandardScaler()
    X_train_reg = scaler_reg.fit_transform(X_train_reg)
    # We don't normalize y_reg for this type of regression / Não normalizamos y_reg para este tipo de regressão
    X_test_reg = scaler_reg.transform(X_test_reg)

In [ ]:
# ===== Função para criar o modelo / Function to create the model =====
def criar_modelo_reg(num_neuronios, input_dim, output_dim):
    model = Sequential([
        Dense(num_neuronios, input_dim=input_dim, activation="relu"),
        Dense(output_dim, activation="linear")
    ])
    model.compile(optimizer="adam", loss="mse", metrics=["mae"])
    return model

In [ ]:
# ===== Avaliação combinada: neurônios X épocas / Combined evaluation: neurons X epoch =====

neuron_range = [2, 4, 8, 16, 32, 64] # quantidade de neurônios a ser testados / number of neurons to be tested
epoch_range = list(range(5, 105, 5))  # de 'início' a 'final' com 'passo' / from 'start' to 'end' with 'step'
mse_results = {n: [] for n in neuron_range}
melhor_mse = float("inf")
melhor_comb = (None, None)

# Check if regression data is available
if X_train_reg is not None and y_train_reg is not None and X_test_reg is not None and y_test_reg is not None:
    for n in neuron_range:
        print(f"\n>> Neurons: {n}")
        for ep in epoch_range:
            # Use the regression model creation function and regression data
            model = criar_modelo_reg(n, X_train_reg.shape[1], y_train_reg.shape[1])
            model.fit(
                X_train_reg, y_train_reg,
                validation_split=0.2,
                epochs=ep,
                batch_size=16,
                verbose=0
            )
            # Predict and calculate MSE using regression test data
            y_pred = model.predict(X_test_reg)
            mse = mean_squared_error(y_test_reg, y_pred)
            mse_results[n].append(mse)

            print(f"   Epochs: {ep} → MSE: {mse:.4f}")

            if mse < melhor_mse:
                melhor_mse = mse
                melhor_comb = (n, ep)

    # ===== Plot MSE vs Épocas =====
    plt.figure(figsize=(10,6))
    for n in neuron_range:
        plt.plot(epoch_range, mse_results[n], marker='o', label=f'{n} neurons')

    plt.title("Mean Squared Error (MSE) vs Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("MSE on the test set")
    plt.legend(title="Hidden layer")
    plt.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print("Regression data (X_train_reg, y_train_reg, X_test_reg, y_test_reg) not found. Please run the previous cells to generate this data.")
    melhor_comb = (None, None) # Reset melhor_comb if data is missing / Redefinir melhor_comb se os dados estiverem faltando
    melhor_mse = float("inf") # Reset melhor_mse as well / Redefinir mrlhor_mse também

In [ ]:
# ===== Resultado melhor combinação / Best combination result =====
melhor_n, melhor_ep = melhor_comb
print(f"\nBest configuration: {melhor_n} neurons and {melhor_ep} epochs (MSE: {melhor_mse:.4f})")

In [ ]:
# ===== Treinar modelo final com melhor combinação / Train final model with best combination =====
# Use the regression model creation function / Use a função de criação do modelo de regressão
model_reg_final = criar_modelo_reg(melhor_n, X_train_reg.shape[1], y_train_reg.shape[1])
history_reg_final = model_reg_final.fit(
    X_train_reg, y_train_reg,
    validation_split=0.2,
    epochs=melhor_ep,
    batch_size=16,
    verbose=1
)

# Avaliar modelo final com métricas de regressão / Evaluate final model with regression metrics
y_pred_reg_final = model_reg_final.predict(X_test_reg)
mse_final = mean_squared_error(y_test_reg, y_pred_reg_final)
r2_final = r2_score(y_test_reg, y_pred_reg_final)


print(f"\nTest Mean Squared Error: {mse_final:.4f}")
print(f"Test R-squared: {r2_final:.4f}")

In [ ]:
# ===== Mostrar e salvar pesos aprendidos / Show and save learned weights =====
for i, layer in enumerate(model_reg_final.layers):
    weights, biases = layer.get_weights()
    layer_name = layer.name

    print(f"\nLayer {i+1} - {layer_name}")
    print("Weights (W):")
    print(weights)
    print("Biases (b):")
    print(biases)

    # Salvar Pesos
    weights_df = pd.DataFrame(weights)
    weights_csv_filename = f"weights_{layer_name}.csv"
    weights_excel_filename = f"weights_{layer_name}.xlsx"

    weights_df.to_csv(weights_csv_filename, index=False)
    weights_df.to_excel(weights_excel_filename, index=False)
    print(f"Layer weights {layer_name} saved in {weights_csv_filename} e {weights_excel_filename}")

    # Salvar Biases (verificar se a camada tem biases) / Save Biases (check if the layer has biases)
    if len(biases) > 0:
        biases_df = pd.DataFrame(biases, columns=['Bias'])
        biases_csv_filename = f"biases_{layer_name}.csv"
        biases_excel_filename = f"biases_{layer_name}.xlsx"

        biases_df.to_csv(biases_csv_filename, index=False)
        biases_df.to_excel(biases_excel_filename, index=False)
        print(f"Layer biases {layer_name} saved in {biases_csv_filename} and {biases_excel_filename}")
    else:
        print(f"Layer {layer_name} has no biases.")

In [ ]:
# Plotting training history for regression / Traçando o histórico de treinamento para regressão
plt.figure(figsize=(8,5))
plt.plot(history_reg_final.history['loss'], label='Train')
plt.plot(history_reg_final.history['val_loss'], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curve - Regression')
plt.legend()
plt.grid(True)
plt.show()

# Regressão / Regression

In [ ]:
target_column_names = ['feature1', 'feature2'] # Change to a list of your desired target columns / Alterar para uma lista de suas colunas de destino desejadas

# Check if all target columns exist in the DataFrame / Verifica se todas as colunas de destino existem no DataFrame
if all(col in df.columns for col in target_column_names):
    y_reg = df[target_column_names].values
    # Define features for regression, excluding the classification target and regression targets / Definir recursos para regressão, excluindo o alvo de classificação e os alvos de regressão
    # Assuming the first column is the classification target / Supondo que a primeira coluna seja o alvo da classificação
    feature_columns_reg = [col for col in df.columns if col not in [df.columns[0]] + target_column_names]
    X_reg = df[feature_columns_reg].values

else:
    missing_cols = [col for col in target_column_names if col not in df.columns]
    print(f"Columns {missing_cols} not found in the DataFrame. Please check the column names.")
    y_reg = None # Set y_reg to None if columns are missing / Define y_reg como None se colunas estiverem faltando
    X_reg = None # Also set X_reg to None if columns are missing / Define também X_reg como None se colunas estiverem faltando


# Split data for regression only if target columns were found / Dividir dados para regressão somente se colunas de destino forem encontradas
if y_reg is not None and X_reg is not None:
    X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
        X_reg, y_reg, test_size=0.2, random_state=42
    )

    # Normalize data for regression / Normalizar dados para regressão
    scaler_reg = StandardScaler()
    X_train_reg = scaler_reg.fit_transform(X_train_reg)
    # We don't normalize y_reg for this type of regression /Não normalizamos y_reg para este tipo de regressão
    X_test_reg = scaler_reg.transform(X_test_reg)

In [ ]:
# Define regression MLP model / Define MLP regression model
neurons = 16 # número de neurônios na camada oculta / number of neurons in the hidden layer
model_reg = Sequential([
    Dense(neurons, input_dim=X_train_reg.shape[1], activation="relu"),
    Dense(len(target_column_names), activation='linear')  # Output layer for regression / Camada de saída para regressão
])

In [ ]:
# Compile regression model / Compilar modelo de regressão
model_reg.compile(optimizer="adam", loss="mse") # Using Mean Squared Error for regression / Usando erro quadrático médio para regressão

# Train regression model / treino do modelo de regressão
history_reg = model_reg.fit(
    X_train_reg, y_train_reg,
    validation_split=0.2,
    epochs=20,
    batch_size=16,
    verbose=1
)

# Evaluate regression model / Avaliar modelo de regressão
y_pred_reg = model_reg.predict(X_test_reg)
mse = mean_squared_error(y_test_reg, y_pred_reg)
r2 = r2_score(y_test_reg, y_pred_reg)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

In [ ]:
# ===== Gráficos de comparação: Real vs Predito (Regressão) / Comparison Charts: Actual vs Predicted (Regression) =====

if y_test_reg is not None and y_pred_reg is not None:
    num_targets = y_test_reg.shape[1]

    plt.figure(figsize=(6 * num_targets, 5)) # Ajusta o tamanho da figura com base no número de alvos / Adjusts the size of the figure based on the number of targets

    for i in range(num_targets):
        plt.subplot(1, num_targets, i + 1)
        plt.scatter(y_test_reg[:, i], y_pred_reg[:, i], alpha=0.5)
        plt.xlabel(f"Real - {target_column_names[i]}")
        plt.ylabel(f"Predicted - {target_column_names[i]}")
        plt.title(f"Actual vs Predicted Comparison for {target_column_names[i]}")
        plt.plot([y_test_reg[:, i].min(), y_test_reg[:, i].max()],
                 [y_test_reg[:, i].min(), y_test_reg[:, i].max()],
                 'k--', lw=2) # Adiciona linha de referência / Add reference line

        # Calculate R-squared for the current target variable / Calcular R-quadrado para a variável alvo atual
        r2_target = r2_score(y_test_reg[:, i], y_pred_reg[:, i])
        plt.text(0.05, 0.95, f'R² = {r2_target:.2f}', transform=plt.gca().transAxes, fontsize=10,
                 verticalalignment='top')


    plt.tight_layout()
    plt.show()
else:
    print("Test data (y_test_reg) or predictions (y_pred_reg) not found. Run the previous codes to generate this data.")

In [ ]:
# Gráfico: Curva Real vs Predita (linha) para cada variável / Graph: Actual vs Predicted Curve (line) for each variable
for i, target in enumerate(target_column_names):
    plt.figure(figsize=(10, 5))
    plt.plot(y_test_reg[:, i], label='Real', marker='o', linestyle='-')
    plt.plot(y_pred_reg[:, i], label='Predicted', marker='x', linestyle='-')
    plt.title(f"Actual vs Predicted Curve - {target}")
    plt.xlabel("Sample Index")
    plt.ylabel(f"Value of {target}")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
